In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus.util import remove_stopwords


In [26]:
rstate=10

In [27]:
df = pd.read_csv("Bengali hate speech.csv")

In [28]:
import string 
def remove_punctuation_from_sentence(sentence):
    sentence_without_punctuation="".join([c for c in sentence if c not in string.punctuation])
    return sentence_without_punctuation

df['sentence without punctuation'] = df['sentence'].apply(lambda x:remove_punctuation_from_sentence(x))
df.head()

,sentence,hate,category,sentence without punctuation
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব


In [29]:
df['Removed Stopped word'] = df['sentence without punctuation'].apply(lambda x: remove_stopwords(x,stopwords))
df.head()

,sentence,hate,category,sentence without punctuation,Removed Stopped word
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,"[যত্তসব, পাপন, শালার, ফাজলামী]"
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,"[পাপন, শালা, রে, রিমান্ডে, দরকার]"
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,"[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,..."
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,"[শালা, লুচ্চা, পাঠার, মত, যায়]"
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,"[তুই, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডায়, খ..."


In [30]:
df['hate_speech']=df['Removed Stopped word'].apply(' '.join)

In [31]:
y=df['hate']

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['hate_speech'])
sequences = tokenizer.texts_to_sequences(df['hate_speech'])
x= pad_sequences(sequences, maxlen=100)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=rstate)

In [34]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=100))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [36]:
model.fit(X_train, y_train, batch_size=32, epochs=42, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy*100.0)

Epoch 1/42
750/750 [==============================] - 52s 67ms/step - loss: 0.3894 - accuracy: 0.8307 - val_loss: 0.3062 - val_accuracy: 0.8767
Epoch 2/42
750/750 [==============================] - 51s 68ms/step - loss: 0.1935 - accuracy: 0.9260 - val_loss: 0.3308 - val_accuracy: 0.8728
Epoch 3/42
750/750 [==============================] - 51s 67ms/step - loss: 0.1081 - accuracy: 0.9621 - val_loss: 0.3876 - val_accuracy: 0.8582
Epoch 4/42
750/750 [==============================] - 51s 68ms/step - loss: 0.0684 - accuracy: 0.9770 - val_loss: 0.4960 - val_accuracy: 0.8535
Epoch 5/42
750/750 [==============================] - 51s 68ms/step - loss: 0.0497 - accuracy: 0.9839 - val_loss: 0.5438 - val_accuracy: 0.8482
Epoch 6/42
750/750 [==============================] - 51s 68ms/step - loss: 0.0384 - accuracy: 0.9867 - val_loss: 0.6060 - val_accuracy: 0.8492
Epoch 7/42
750/750 [==============================] - 51s 69ms/step - loss: 0.0314 - accuracy: 0.9889 - val_loss: 0.5243 - val_accuracy:

In [37]:
#Saving the model
model_json=model.to_json()
with open("bangla-sequential-model.json","w")as json_file:
    json_file.write(model_json)
model.save_weights("bangla-sequential-model_wieghts.h5")